# TorQ: Create Everything
This notebook will use the AWS boto3 APIs to create the needed resources for a TorQ based application. The notebook will first clone the relevant gihub code (TorQ and TorQ AMazon FinSpace Starter Pack) then proceed to create the necessary AWS resources. 

Once you have create all clusters, you can see how to query for data through the gateway, see the [pykx_query_all](pykx_query_all.ipynb) notebook

To cleanup (delete) all resources, run the [delete_all](delete_all.ipynb) notebook.

## AWS Resources Created
- Database   
- Changeset to add data to database   
- Scaling Group that will contain all clusters   
- Shared Volume   
- Dataview of database on the shared volume   
- Clusters

This notebook is based on the TorQ Amazon FinSpace starter pack but uses Scaling Groups and Shared Volumes for cost savings.

[TorQ Amazon FinSpace Starter Pack](https://dataintellecttech.github.io/TorQ-Amazon-FinSpace-Starter-Pack/)

### Branches

**TorQ-Amazon-FinSpace-Starter-Pack**: v1.0.2   
**TorQ**: v5.0.3

**Note**: For other branches, be sure to update the git clone statements below.

In [1]:
!rm -rf torq_app*.zip 

In [2]:
!rm -rf TorQ TorQ-Amazon-FinSpace-Starter-Pack

In [3]:
!git -c advice.detachedHead=false clone --depth 1 --branch  v1.0.2 https://github.com/DataIntellectTech/TorQ-Amazon-FinSpace-Starter-Pack.git 

Cloning into 'TorQ-Amazon-FinSpace-Starter-Pack'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 228 (delta 41), reused 181 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (228/228), 35.79 MiB | 33.38 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [4]:
!git -c advice.detachedHead=false clone --depth 1 --branch v5.0.3 https://github.com/DataIntellectTech/TorQ.git 

Cloning into 'TorQ'...
remote: Enumerating objects: 797, done.
remote: Counting objects: 100% (797/797), done.
remote: Compressing objects: 100% (550/550), done.
remote: Total 797 (delta 195), reused 588 (delta 174), pack-reused 0 (from 0)
Receiving objects: 100% (797/797), 5.63 MiB | 25.96 MiB/s, done.
Resolving deltas: 100% (195/195), done.


In [5]:
!ln -sf ../finspace_torq.q TorQ-Amazon-FinSpace-Starter-Pack
# this is the one modification over what is in the starter-pack on github

In [6]:
import os
import subprocess
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *

from env import *
from config import *

# ----------------------------------------------------------------

TORQ_CODEBASE="TorQ"
TORQ_FINSPACE_CODEBASE="TorQ-Amazon-FinSpace-Starter-Pack"

# Source data directory
SOURCE_DATA_DIR=f"{TORQ_FINSPACE_CODEBASE}/hdb"

# Code directory
CODEBASE="torq_app"

# S3 Destinations
S3_CODE_PATH="code"
S3_DATA_PATH="data"

NODE_TYPE="kx.sg.xlarge"

DATABASE_CONFIG=[{ 
    'databaseName': DB_NAME,
    'dataviewName': DBVIEW_NAME
    }]
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }

NAS1_CONFIG= {
        'type': 'SSD_250',
        'size': 1200
}

VPC_CONFIG={ 
    'vpcId': VPC_ID,
    'securityGroupIds': SECURITY_GROUPS,
    'subnetIds': SUBNET_IDS,
    'ipAddressType': 'IP_V4' 
}

In [7]:
# Using credentials and create service client
session = boto3.Session()

# create finspace client
client = session.client(service_name='finspace')

# Create the Database
Create a database from the supplied data in hdb.tar.gz.  

## Stage HDB Data on S3
Using AWS cli, copy hdb to staging bucket

In [8]:
S3_DEST=f"s3://{S3_BUCKET}/{S3_DATA_PATH}/{SOURCE_DATA_DIR}/"

if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

aws s3 sync --quiet --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
else:
    cp = f"""
aws s3 sync --quiet --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
    
# execute the S3 copy
os.system(cp)

                           PRE 2014.04.21/
                           PRE 2014.04.22/
                           PRE 2014.04.23/
2024-12-03 19:44:34         57 sym


0

## Create Managed Database
Using the AWS APIs, create a managed database in Managed kdb Insights.

In [9]:
# assume it exists
create_db=False

try:
    resp = client.get_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)
    resp.pop('ResponseMetadata', None)
except:
    # does not exist, will create
    create_db=True

if create_db:
    print(f"CREATING Database: {DB_NAME}")
    resp = client.create_kx_database(environmentId=ENV_ID, databaseName=DB_NAME, description="Basictick kdb database")
    resp.pop('ResponseMetadata', None)

    print(f"CREATED Database: {DB_NAME}")

print(json.dumps(resp,sort_keys=True,indent=4,default=str))

CREATING Database: finspace-database
CREATED Database: finspace-database
{
    "createdTimestamp": "2024-12-03 19:44:35.065000+00:00",
    "databaseArn": "arn:aws:finspace:us-east-1:829845998889:kxEnvironment/jlcenjvtkgzrdek2qqv7ic/kxDatabase/finspace-database",
    "databaseName": "finspace-database",
    "description": "Basictick kdb database",
    "environmentId": "jlcenjvtkgzrdek2qqv7ic",
    "lastModifiedTimestamp": "2024-12-03 19:44:35.065000+00:00"
}


## Add HDB Data to Database
Add the data in the local hdb directory to the managed database using the changeset mechanism. The Data will be copied to S3 then ingested with the create-kx-changeset API.

In [10]:
c_set_list = list_kx_changesets(client, environmentId=ENV_ID, databaseName=DB_NAME)
len(c_set_list)

0

In [11]:
# TODO: check is there is a changeset in the database, if so, no need to add another
c_set_list = list_kx_changesets(client, environmentId=ENV_ID, databaseName=DB_NAME)

if len(c_set_list) == 0:
    # if changesets exist, set chagnset_id to last created one

    changes=[]

    for f in os.listdir(f"{SOURCE_DATA_DIR}"):
        if os.path.isdir(f"{SOURCE_DATA_DIR}/{f}"):
            changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}/", 'dbPath': f"/{f}/" } )
        else:
            changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}", 'dbPath': f"/" } )

    resp = client.create_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, 
        changeRequests=changes)

    resp.pop('ResponseMetadata', None)
    changeset_id = resp['changesetId']

    print("Changeset...")
    print(json.dumps(resp,sort_keys=True,indent=4,default=str))
else:
    c_set_list=sorted(c_set_list, key=lambda d: d['createdTimestamp']) 
    changeset_id=c_set_list[-1]['changesetId']
    print(f"Using Last changeset: {changeset_id}")


Changeset...
{
    "changeRequests": [
        {
            "changeType": "PUT",
            "dbPath": "/2014.04.23/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.23/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2014.04.22/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.22/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/sym"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2014.04.21/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.21/"
        }
    ],
    "changesetId": "2MnHBswHfdRpfkJBJVMANw",
    "createdTimestamp": "2024-12-03 19:44:35.856000+00:00",
    "databaseName": "finspace-database",
    "environmentId":

In [12]:
wait_for_changeset_status(client, environmentId=ENV_ID, databaseName=DB_NAME, changesetId=changeset_id, show_wait=True)
print("**Done**")

Status is IN_PROGRESS, total wait 0:00:00, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:10, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:20, waiting 10 sec ...
**Done**


In [13]:
note_str = ""

c_set_list = list_kx_changesets(client, environmentId=ENV_ID, databaseName=DB_NAME)

if len(c_set_list) == 0:
    note_str = "<<Could not get changesets>>"
    
print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"  Changeset: {c_set_id}: Created: {c['createdTimestamp']} ({c['status']})")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

Database: finspace-database, Changesets: 1 
  Changeset: 2MnHBswHfdRpfkJBJVMANw: Created: 2024-12-03 19:44:35.856000+00:00 (COMPLETED)


changeType,s3Path,dbPath
PUT,s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.23/,/2014.04.23/
PUT,s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.22/,/2014.04.22/
PUT,s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/sym,/
PUT,s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.21/,/2014.04.21/


# Create Scaling Group
The scaling group represents the total compute avilable to the application. All clusters will be placed into the scaling group ans share the compute and memory of the scaling group.

In [14]:
# Check if scaling group exits, only create if it does not
resp = get_kx_scaling_group(client=client, environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME)

if resp is None:
    resp = client.create_kx_scaling_group(
        environmentId = ENV_ID, 
        scalingGroupName = SCALING_GROUP_NAME,
        hostType=NODE_TYPE,
        availabilityZoneId = AZ_ID
    )
else:
    print(f"Scaling Group {SCALING_GROUP_NAME} exists")

In [15]:
resp

{'ResponseMetadata': {'RequestId': '8fae45fa-a679-4d72-bf5c-d08a4f8a902b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '237',
   'connection': 'keep-alive',
   'date': 'Tue, 03 Dec 2024 19:45:11 GMT',
   'x-amzn-requestid': '8fae45fa-a679-4d72-bf5c-d08a4f8a902b',
   'x-amz-apigw-id': 'COvm1ErOIAMEG2Q=',
   'x-amzn-trace-id': 'Root=1-674f5fc4-3ca0c0cc723507f97a1f2064',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2174e600dd54879ba9f49d0337eeb2dc.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'nMZmwBT6_ZZvmJ_VtVOzfZJCQaYULYTwjCr_uyptzofKaxnns6VJig=='},
  'RetryAttempts': 0},
 'environmentId': 'jlcenjvtkgzrdek2qqv7ic',
 'scalingGroupName': 'SCALING_GROUP_torq',
 'hostType': 'kx.sg.xlarge',
 'availabilityZoneId': 'use1-az6',
 'status': 'CREATING',
 'lastModifiedTimestamp': datetime.datetime(2024, 12, 3, 19, 45, 10, 883000, tzinfo=tzlocal()),
 'createdTimestamp': datetime.datetime(2024, 12, 3, 19

# Create Shared Volume
The shared volume is a common storage device for the application. Every cluster using the shared volume will have a writable directory named after the cluster, can read the directories named after other clusters in the application using the volume. Also, there is a common 

In [16]:
# Check if volume already exists before trying to create one
resp = get_kx_volume(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME)

if resp is None:
    resp = client.create_kx_volume(
        environmentId = ENV_ID, 
        volumeType = 'NAS_1',
        volumeName = VOLUME_NAME,
        description = 'Shared volume between TP and RDB',
        nas1Configuration = NAS1_CONFIG,
        azMode='SINGLE',
        availabilityZoneIds=[ AZ_ID ]    
    )
else:
    print(f"Volume {VOLUME_NAME} exists")    

In [17]:
resp

{'ResponseMetadata': {'RequestId': 'eaf7e431-4f46-4533-95e1-f421588144b7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '431',
   'connection': 'keep-alive',
   'date': 'Tue, 03 Dec 2024 19:45:15 GMT',
   'x-amzn-requestid': 'eaf7e431-4f46-4533-95e1-f421588144b7',
   'x-amz-apigw-id': 'COvnTHSKoAMEaGg=',
   'x-amzn-trace-id': 'Root=1-674f5fc7-572cf2f6707f24e716735df3',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2174e600dd54879ba9f49d0337eeb2dc.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'le2T_WMpJHOtSEkdHcKMALw1tlpVU4kHxJar2FyoxVvh6v_MQkKPLw=='},
  'RetryAttempts': 0},
 'environmentId': 'jlcenjvtkgzrdek2qqv7ic',
 'volumeName': 'SHARED_torq',
 'volumeType': 'NAS_1',
 'volumeArn': 'arn:aws:finspace:us-east-1:829845998889:kxEnvironment/jlcenjvtkgzrdek2qqv7ic/kxVolume/SHARED_torq',
 'nas1Configuration': {'type': 'SSD_250', 'size': 1200},
 'status': 'CREATING',
 'azMode': 'SINGLE',
 'descript

# Wait for Volume and Scaling Group
Before proceeding to use Volumes and Scaling groups, wait for their creation to complete.

Volume will be used by the dataview.    
Dataview and Scaling Group will be used by the clusters


In [18]:
# wait for the scaling group to create
wait_for_scaling_group_status(client=client, environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME, show_wait=True)
print("** DONE **")

# wait for the volume to create
wait_for_volume_status(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME, show_wait=True)
print("** DONE **")

Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:00:00, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:00:30, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:01:00, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:01:30, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:02:00, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:02:30, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:03:00, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:03:30, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:04:00, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:04:30, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is now ACTIVE, total wait 0:0

# Create Dataview
Create a dataview, for a specific (static) version of the database and have all of its data cached using the shared volume.

In [19]:
# Check if dataview already exists and is set to the requested changeset_id
resp = get_kx_dataview(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME)

if resp is None:
    # sort changeset list by create time
    c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

    resp = client.create_kx_dataview(
        environmentId = ENV_ID, 
        databaseName=DB_NAME, 
        dataviewName=DBVIEW_NAME,
        azMode='SINGLE',
        availabilityZoneId=AZ_ID,
        changesetId=c_set_list[-1]['changesetId'],
        segmentConfigurations=[
            { 
                'dbPaths': ['/*'],
                'volumeName': VOLUME_NAME
            }
        ],
        autoUpdate=False,
        description = f'Dataview of database'
    )
else:
    print(f"Dataview {DBVIEW_NAME} exists")        

In [20]:
# wait for the view to create
wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME, show_wait=True)
print("** DONE **")

Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:00:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:00:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:01:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:01:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:02:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:02:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:03:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:03:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:04:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:04:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total

# Create Clusters
With foundation resources now completed, create the needed clusters for the application.

## Stage Code to S3
Code to be used in this application must be staged to an S3 bucket the service can read from, that code will then be deployed to the clusters as part of their creation workflow.

In [21]:
# zip the code
os.system(f"zip -q -r {CODEBASE}.zip {TORQ_CODEBASE}/ {TORQ_FINSPACE_CODEBASE}/ -x '*.ipynb_checkpoints*' -x '*/hdb/*' -x '*.git*' -x '*/tests/*' -x '*/terraform-deployment/*' -x '*/docs/*' -x '*/lib/*' -x '*/html/*' -x '*/datadog/*'  -x '*/monit/*'")

cp = ""

# copy code to S3
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}
"""

cp += f"""
aws s3 cp --exclude .DS_Store {CODEBASE}.zip s3://{S3_BUCKET}/code/{CODEBASE}.zip
aws s3 ls s3://{S3_BUCKET}/code/{CODEBASE}.zip
"""

# execute the S3 copy
os.system(cp)

upload: ./torq_app.zip to s3://kdb-demo-829845998889-kms/code/torq_app.zip
2024-12-03 20:01:30     486888 torq_app.zip


0

In [22]:
for c in clusters:
    # wait for a cluster?
    if c['type'] == "WAIT":
        wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=c['name'], show_wait=True)
        continue

    cluster_name = c['name']
    cluster_type = c['type']
    cluster_init = c['init']
    cluster_args = c['args']

    # cluster already exists
    resp = get_kx_cluster(client, environmentId=ENV_ID, clusterName=cluster_name)
    if resp is not None:
        print(f"Cluster: {cluster_name} already exists")
        continue

    print(f"Creating: {cluster_name}")

    resp = client.create_kx_cluster(
        environmentId=ENV_ID, 
        clusterName=cluster_name,
        clusterType=cluster_type,
        releaseLabel = '1.0',
        executionRole=EXECUTION_ROLE,
        databases=DATABASE_CONFIG,
        scalingGroupConfiguration={
            'memoryReservation': 6,
            'nodeCount': 1,
            'scalingGroupName': SCALING_GROUP_NAME,
        },
        savedownStorageConfiguration ={ 'volumeName': VOLUME_NAME },
        clusterDescription="Created with create_all notebook",
        code=CODE_CONFIG,
        initializationScript=cluster_init,
        commandLineArguments=cluster_args,
        azMode=AZ_MODE,
        availabilityZoneId=AZ_ID,
        vpcConfiguration=VPC_CONFIG
    )

    display(resp)

Creating: discovery1


{'ResponseMetadata': {'RequestId': '9707ab7a-a90c-4301-a924-7a89f11665ea',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1675',
   'connection': 'keep-alive',
   'date': 'Tue, 03 Dec 2024 20:01:36 GMT',
   'x-amzn-requestid': '9707ab7a-a90c-4301-a924-7a89f11665ea',
   'x-amz-apigw-id': 'COyAKG5KoAMEqUA=',
   'x-amzn-trace-id': 'Root=1-674f639a-3d8d59df232d653156b73bcb',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2174e600dd54879ba9f49d0337eeb2dc.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'FJtqQW8JhAuRCTkhuP_HW1RfVg0YKWQk3Qh4Gu8pAqnV1NXNSebsGQ=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'discovery1',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'SHARED_torq', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'finspace-database',
   'cacheConfigurations': [],
   'dataviewConfiguration': {'dataviewName': 'finspace-database_DBVIEW',
    'dataviewVersionId': 

Creating: rdb1


{'ResponseMetadata': {'RequestId': '5d143ee5-9ac9-4f66-b20f-5d962a20ca39',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1657',
   'connection': 'keep-alive',
   'date': 'Tue, 03 Dec 2024 20:01:39 GMT',
   'x-amzn-requestid': '5d143ee5-9ac9-4f66-b20f-5d962a20ca39',
   'x-amz-apigw-id': 'COyBIF2ooAMEtrA=',
   'x-amzn-trace-id': 'Root=1-674f63a0-5b7cf8f1694fdafe0ddd9e87',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2174e600dd54879ba9f49d0337eeb2dc.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': '77h8oE1aIZrLTFjB-DhJPd7nmC1wVIbXBMlKkqWQhSzKTXfEg1cbgg=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'rdb1',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'SHARED_torq', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'finspace-database',
   'cacheConfigurations': [],
   'dataviewConfiguration': {'dataviewName': 'finspace-database_DBVIEW',
    'dataviewVersionId': 'jsnHC

Creating: hdb1


...

{'ResponseMetadata': {'RequestId': '15ab8f2e-8a08-469f-b0eb-ed54dcc66af7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1669',
   'connection': 'keep-alive',
   'date': 'Tue, 03 Dec 2024 20:18:24 GMT',
   'x-amzn-requestid': '15ab8f2e-8a08-469f-b0eb-ed54dcc66af7',
   'x-amz-apigw-id': 'CO0eAE7hoAMEtnQ=',
   'x-amzn-trace-id': 'Root=1-674f678c-22f328347fb1d17345bae243',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2174e600dd54879ba9f49d0337eeb2dc.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'Zb-_qPIFv61cf26lMI7yc7vwNsHdIATbNENaeEg0VAQlnkajX-IHbA=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'gateway1',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'SHARED_torq', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'finspace-database',
   'cacheConfigurations': [],
   'dataviewConfiguration': {'dataviewName': 'finspace-database_DBVIEW',
    'dataviewVersionId': 'j

Creating: feed1


{'ResponseMetadata': {'RequestId': 'fda47443-1942-401f-9502-dc53985704e9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1670',
   'connection': 'keep-alive',
   'date': 'Tue, 03 Dec 2024 20:18:27 GMT',
   'x-amzn-requestid': 'fda47443-1942-401f-9502-dc53985704e9',
   'x-amz-apigw-id': 'CO0euFWZoAMEu2w=',
   'x-amzn-trace-id': 'Root=1-674f6791-727eca3d173aa6eb1f2abc16',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2174e600dd54879ba9f49d0337eeb2dc.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'HHSOYWC2sGWwUSCUHQOPSZuoclNp9H1W4Ap2kIFzx0RxPufEiHi7jw=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'feed1',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'SHARED_torq', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'finspace-database',
   'cacheConfigurations': [],
   'dataviewConfiguration': {'dataviewName': 'finspace-database_DBVIEW',
    'dataviewVersionId': 'jsnH

Creating: monitor1


{'ResponseMetadata': {'RequestId': '881e0ee9-1511-46b3-a33e-35b4d9815225',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1669',
   'connection': 'keep-alive',
   'date': 'Tue, 03 Dec 2024 20:18:31 GMT',
   'x-amzn-requestid': '881e0ee9-1511-46b3-a33e-35b4d9815225',
   'x-amz-apigw-id': 'CO0fNFJLIAMER4A=',
   'x-amzn-trace-id': 'Root=1-674f6794-0d4bdb907ecf68fa7812757d',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2174e600dd54879ba9f49d0337eeb2dc.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 's2N6yEZtETnm1621l2qmuMnLRUgRZMDlajtgeP4l5RPBhwKx1i6u6w=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'monitor1',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'SHARED_torq', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'finspace-database',
   'cacheConfigurations': [],
   'dataviewConfiguration': {'dataviewName': 'finspace-database_DBVIEW',
    'dataviewVersionId': 'j

## Wait for all clusters to finish creating

In [23]:
# Wait for all clusters to start
for c in clusters:
    cluster_name = c['name']
    wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=cluster_name, show_wait=True)

print("** ALL DONE **")

Cluster: discovery1 status is now RUNNING, total wait 0:00:00
Cluster: rdb1 status is now RUNNING, total wait 0:00:00
Cluster: hdb1 status is now RUNNING, total wait 0:00:00
Cluster: discovery1 status is now RUNNING, total wait 0:00:00
Cluster: rdb1 status is now RUNNING, total wait 0:00:00
Cluster: gateway1 status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: gateway1 status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: gateway1 status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: gateway1 status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: gateway1 status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: gateway1 status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: gateway1 status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: gateway1 status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: gateway1 status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: gateway

# List Clusters

In [24]:
cdf = get_clusters(client, environmentId=ENV_ID)

all_clusters = [d['name'] for d in clusters if 'name' in d]

if cdf is not None:
    cdf = cdf[cdf['clusterName'].isin(all_clusters)]

display(cdf)

,clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
3,discovery1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'discovery'}, {'key': 'procname', 'value': 'discovery1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-12-03 20:16:25.763000+00:00,2024-12-03 20:01:35.746000+00:00,finspace-database,None
4,feed1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'tradeFeed'}, {'key': 'procname', 'value': 'tradeFeed1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-12-03 20:30:55.001000+00:00,2024-12-03 20:18:27.837000+00:00,finspace-database,None
5,gateway1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'gateway'}, {'key': 'procname', 'value': 'gateway1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-12-03 20:30:55.616000+00:00,2024-12-03 20:18:24.649000+00:00,finspace-database,None
6,hdb1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'hdb'}, {'key': 'procname', 'value': 'hdb1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '4'}]",Created with create_all notebook,2024-12-03 20:17:05.878000+00:00,2024-12-03 20:01:41.735000+00:00,finspace-database,None
7,monitor1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'monitor'}, {'key': 'procname', 'value': 'monitor1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '1'}]",Created with create_all notebook,2024-12-03 20:29:55.787000+00:00,2024-12-03 20:18:30.995000+00:00,finspace-database,None
8,rdb1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'rdb'}, {'key': 'procname', 'value': 'rdb1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-12-03 20:18:04.815000+00:00,2024-12-03 20:01:39.007000+00:00,finspace-database,None


# All Processes Running

In [25]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-12-03 20:31:09.723819
